In [1]:
import pandas as pd
import datetime
from pandas.tseries.offsets import BDay
from dateutil.relativedelta import relativedelta
from collections import defaultdict 
import pprint

from commons import read_xls, get_stock_data, NSE_BSE, BSE_NSE, save_csv, get_holding_quantities
from commons import min_consecutive_sum_kadane, get_curr_prices_from_holdings

In [2]:
HOLDINGS_FILE_NAME = '../docs/Holdings_28Dec23.xlsx'

In [3]:
def process_sheets(df, today=None):
    if not today:
        today = datetime.datetime.today()
        
    start = today - BDay(1) # Get prev business date as market didnt start today
    end = today

#     if datetime.time(today.hour, today.minute, today.second) < datetime.time(9, 15, 5):
#         print('Markets didnt start yet, getting data from prev business dates')
#         end = start
#         start -= BDay(2)
    
    try:
        print(f'****** Calling for stock data between {start} and {end} *******')

        curr_prices = get_stock_data(start=start, end=end, print_data=False)
        curr_prices = curr_prices.reset_index().transpose().reset_index().rename(columns={0: 'currPrice'}) #.rename(columns={'index': 'Symbol'})
    except e: 
        print(e)
        print('***** reading current prices from Holdings.txt as yahoo crapped again!!!')
        curr_prices = get_curr_prices_from_holdings(HOLDINGS_FILE_NAME)
        
    df = df.merge(curr_prices, left_on=['COMPANY'], right_on=['index'])
    df = df.drop('index', axis=1)
    df = df.sort_values(by='Date', ascending=True)
    df['PnL'] = (df['currPrice'] - df['BuyPrice']) * df['Qty']

#     def flip_sells(t, val):
#         if t == 'sell':
#             val *= -1
            
#         return val

#     df['PnL']      = df.apply(lambda x: flip_sells(x['Trade Type'], x['PnL']), axis=1)
#     df['Quantity'] = df.apply(lambda x: flip_sells(x['Trade Type'], x['Quantity']), axis=1)
    return df

def get_stcl(df, harvest_loss_amount=9999999, today=datetime.datetime.today(), num_years_for_stcg=1):
    '''
        Get the number qty to sell to harvest Short Term Cap Loss of a sum X
    '''
    stcg_cutoff_date = (today - relativedelta(years=num_years_for_stcg)).date()
#     df = df[df['Trade Date'] > str(date)] # Get trades after this date
    df = df.sort_values(by='Date', ascending=True)
    data = {}
    
    for c in NSE_BSE.keys():
        df_c = df[df['COMPANY'] == c]
        df_dict = df_c.to_dict('list') # https://stackoverflow.com/questions/52547805/how-to-convert-dataframe-to-dictionary-in-pandas-without-index
        ltcg, stcg, ltcg_qty, stcg_qty = 0, 0, 0, 0
        data[c] = defaultdict(int)
        
        for i, (comp, dt, bp, qty, cp, pnl) in enumerate(zip(df_dict['COMPANY'], df_dict['Date'], df_dict['BuyPrice'], \
                                             df_dict['Qty'], df_dict['currPrice'], df_dict['PnL'])):
            i += 1 
            
            if dt.date() <= stcg_cutoff_date:
                ### LTCG
#                 print(f'LTCG {i} {dt.date()} >= {stcg_cutoff_date}: {c} Qty={qty} buyPrice={bp} currPrice={cp} pnl={pnl}')
                data[c]['ltcg_qty'] += qty
                data[c]['ltcg'] += pnl
            else:
                ### STCG
#                 print(f'STCG {i} {dt.date()} <  {stcg_cutoff_date}: {c} Qty={qty} buyPrice={bp} currPrice={cp} pnl={pnl}')
                
                # STCLoss only if prev last LTCG txn was negative
                if 'stcg_qty' not in data[c]: # init stcg total qty 
                    data[c]['stcl_qty'] = data[c]['ltcg_qty']
                    data[c]['stcg_qty'] = data[c]['ltcg_qty']

                if pnl < 0: ## only if it is a loss
                    data[c]['stcl_qty'] += qty
                    data[c]['stcl'] += pnl
                else:
                    data[c]['stcg_qty'] += qty
                    data[c]['stcg'] += pnl                    
                
            data[c]['total_qty'] += qty
            data[c]['total'] += pnl            

        
        
        
#     for s in QTYS.keys():
#         s_df = df[df['Symbol'] == s].sort_values(by='Trade Date', ascending=True)
#         print(s_df.to_dict())
#         break
    
    return data

def get_max_stcl(df, harvest_loss_amount=9999999, today=datetime.datetime.today(), num_years_for_stcg=1):
    '''
        Get the number qty to sell to harvest Short Term Cap Loss of a sum X
    '''
    stcg_cutoff_date = (today - relativedelta(years=num_years_for_stcg)).date()
    data = {}
    
    for c in NSE_BSE.keys():
        df_c = df[(df['Date'] > str(stcg_cutoff_date)) & (df['COMPANY'] == c)] # Get trades after this date
        df_dict = df_c.to_dict('list')
#         print(c, df_dict['PnL'])
        data[c] = min_consecutive_sum_kadane(df_dict['PnL'], df_dict['Date'])

    return {k: v for k, v in sorted(data.items(), key=lambda item: item[1])} # sort data by value before returning



In [4]:
df = read_xls(HOLDINGS_FILE_NAME, tab=5)
df = df[df['COMPANY'].isin(get_holding_quantities().keys())]
df = df[['COMPANY', 'Date', 'Qty', 'BuyPrice']]

df = process_sheets(df)
# save_csv(df, 'ltcg_stcg')
max_data = get_max_stcl(df)
data = get_stcl(df)
print(max_data)
data


****** Calling for stock data between 2024-01-19 02:57:03.103712 and 2024-01-20 02:57:03.103712 *******
****** Getting stock data between 2024-01-19 02:57:03.103712 and 2024-01-20 02:57:03.103712 *******
[*********************100%%**********************]  25 of 25 completed


3 Failed downloads:
['MOLDTEK.BO', 'DEEPAKNI.BO', 'RAJGLOWIR.BO']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2024-01-19 02:57:03.103712 -> 2024-01-20 02:57:03.103712)')



****** Got stock data between 2024-01-19 02:57:03.103712 and 2024-01-20 02:57:03.103712 *******
**********                         Adj Close                                          \
                        AFFLE.BO BAJFINANCE.BO      BEL.BO BERGEPAINT.BO   
Date                                                                       
2024-01-19 00:00:00  1263.150024   7320.350098  191.800003    579.549988   

                                                                       \
                    DEEPAKNI.BO  DEVYANI.BO  DIVISLAB.BO     DIXON.BO   
Date                                                                    
2024-01-19 00:00:00         NaN  182.699997  3692.600098  6101.850098   

                                               ...     Volume            \
                        DMART.NS   FINEORG.BO  ... MOLDTEK.BO NAUKRI.BO   
Date                                           ...                        
2024-01-19 00:00:00  3731.699951  4580.600098  ...        NaN      8

{'BAJFINANCE': defaultdict(int,
             {'ltcg_qty': 18,
              'ltcg': 17535.3017578125,
              'total_qty': 51,
              'total': 24736.85498046875,
              'stcl_qty': 32,
              'stcg_qty': 37,
              'stcg': 9994.65185546875,
              'stcl': -2793.0986328125}),
 'DMART': defaultdict(int,
             {'ltcg_qty': 36,
              'ltcg': -16147.8017578125,
              'total_qty': 83,
              'total': -12040.904052734375,
              'stcl_qty': 48,
              'stcg_qty': 71,
              'stcg': 5180.498291015625,
              'stcl': -1073.6005859375}),
 'DEEPAKNTR': defaultdict(int,
             {'ltcg_qty': 56,
              'ltcg': nan,
              'total_qty': 132,
              'total': nan,
              'stcl_qty': 56,
              'stcg_qty': 132,
              'stcg': nan}),
 'SONACOMS': defaultdict(int,
             {'ltcg_qty': 234,
              'ltcg': -1264.6971435546875,
              'total_qty'

In [5]:
# df[['Symbol', 'Quantity', 'PnL']].groupby(['Symbol']).agg('sum')
data['FINEORG'][2].date().strftime('%d%b%Y')

isinstance(data['FINEORG'][2], pd.Timestamp)

AttributeError: 'int' object has no attribute 'date'

In [ ]:
df = read_xls('../docs/Holdings_16Dec23.xlsx', tab=5)
df = df[df['COMPANY'].isin(get_holding_quantities().keys())]
df = df[['COMPANY', 'Date', 'Qty', 'BuyPrice']]
df = process_sheets(df)
# df_dict = df.to_dict('list') # https://stackoverflow.com/questions/52547805/how-to-convert-dataframe-to-dictionary-in-pandas-without-index
# df_dict
df

In [ ]:
# print(s_df)
# s_df.to_dict()
# sum(s_df['Quantity'])
s_df['PnL'] = (s_df['currPrice'] - s_df['Price']) * s_df['Quantity']
print(s_df)
print(sum(s_df[s_df['Trade Type'] == 'buy']['PnL']) - sum(s_df[s_df['Trade Type'] == 'sell']['PnL']))

def f(t, pnl):
    if t == 'sell':
        return -1*pnl
    return pnl

s_df['PnL'] = s_df.apply(lambda x: f(x['Trade Type'], x['PnL']), axis=1)
s_df
# s_df['Trade Type', 'PnL'].apply(lambda x, y: f(x, y))
# s_df[['Trade Type', 'PnL']]

In [ ]:
datetime.datetime.today().minute
# datetime.datetime.today() - BDay(1)
t = datetime.time(9, 30, 59)
t.hour
# datetime.time(datetime.datetime.today())
datetime.time(datetime.datetime.today().hour, datetime.datetime.today().minute) < datetime.time(9, 15, 5)
datetime.datetime.today().second

today = datetime.datetime.today()
today - BDay(2), today - BDay(1) # Get prev business date as market didnt start today

In [ ]:
curr_prices = get_curr_prices_from_holdings()
# print(curr_prices)
df = df.merge(curr_prices, left_on=['COMPANY'], right_on=['index'])
df = df.drop('index', axis=1)
df = df.sort_values(by='Date', ascending=True)
df['PnL'] = (df['currPrice'] - df['BuyPrice']) * df['Qty']
df